In [ ]:
!pip install selenium==3.14.0
!apt-get update 
!apt install chromium-chromedriver

In [ ]:
%cd "/content/drive/MyDrive/KHDL"

/content/drive/MyDrive/KHDL


In [ ]:
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
import csv
import pandas as pd
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
PATH = "chromedriver"
hacom = "https://www.hanoicomputer.vn/laptop"
phong_vu = "https://phongvu.vn/laptop"
anphat = "https://www.anphatpc.com.vn/may-tinh-xach-tay-laptop.html"

#Crawl dữ liệu từ trang web Hà Nội Computer

In [ ]:
def get_url_page(base_url, num):
    if num == 1:
        return base_url
    return base_url + "/" + str(num) + "/"

In [ ]:
def crawl_data(first_driver, second_driver, base_url):
    num_page = 1
    list_url_crawl = []
    list_name = []
    while (1):
        print("Page: ", num_page)
        first_driver.get(get_url_page(base_url, num_page))
        time.sleep(5)

        total_height = int(first_driver.execute_script("return document.body.scrollHeight"))

        for i in range(1, total_height, 100):
            first_driver.execute_script("window.scrollTo(0, {});".format(i))
        items = first_driver.find_elements_by_class_name("p-component")
        if len(items) == 0:
            break
        for item in items:
            print("Crawl ", items.index(item))
            pre_name = item.find_element_by_css_selector("h3.p-name")
            name = pre_name.find_element_by_css_selector("a").text
            url_item = pre_name.find_element_by_css_selector("a").get_attribute("href")
            second_driver.get(url_item)
            time.sleep(2)
            second_height = int(second_driver.execute_script("return document.body.scrollHeight") / 10)

            for i in range(1, second_height, 100):
                second_driver.execute_script("window.scrollTo(0, {});".format(i))
            
            more_items = second_driver.find_elements_by_class_name("san-pham-list")
            if len(more_items) > 0:
                for more_item in more_items:
                    more_name = more_item.find_element_by_css_selector("a").text
                    more_link = more_item.find_element_by_css_selector("a").get_attribute("href")
                    list_name.append(more_name)
                    list_url_crawl.append(more_link)
            list_name.append(name)
            list_url_crawl.append(url_item)
        num_page += 1
            
    return list_name, list_url_crawl

In [ ]:
first_driver = webdriver.Chrome(PATH,chrome_options=chrome_options)
first_driver.maximize_window()

second_driver = webdriver.Chrome(PATH,chrome_options=chrome_options)
second_driver.maximize_window()
    
name, link = crawl_data(first_driver, second_driver, hacom)
data = pd.DataFrame({"Name" : name, "Link" : link})
data.to_csv("Crawl_Search_HN_CPT.csv", sep="\t", index=None)

#Crawl dữ liệu từ Phong Vũ

In [ ]:
def get_url(base_url, num):
    url = base_url + "&page=" + str(num) 
    return url

In [ ]:
def crawl_phongvu(driver, base_url):
    num_page = 1
    list_url = []
    list_name = []
    while (1):
        driver.get(get_url(base_url, num_page))
        time.sleep(2)

        total_height = int(driver.execute_script("return document.body.scrollHeight"))

        for i in range(1, total_height, 100):
            driver.execute_script("window.scrollTo(0, {});".format(i))
        if num_page > 2: 
            print("error page")
            break
        else:
            items = driver.find_elements_by_class_name("css-1xhjahy")
            print("Page: ", num_page)
            for item in items:
                print("Crawl " + str(items.index(item)) + " of " + str(len(items)))
                try:
                    name = item.find_element_by_css_selector(".css-1h67xpm").text
                except:
                    name = item.find_element_by_css_selector(".css-3z5569").text
                list_name.append(name)
                link = item.find_element_by_css_selector("a").get_attribute("href")
                list_url.append(link)
            num_page += 1
    return list_name, list_url
        

#Crawl dữ liệu từ An Phát

In [ ]:
def get_url_anphat(base_url, num_page):
    if num_page == 1:
        return base_url
    return base_url + "?page=" + str(num_page)

In [ ]:
def crawl_anphat(driver, base_url):
    num_page = 1
    list_url = []
    list_name = []
    while (1):
        if num_page > 17:
            print("error page")
            break
        print("Page: ", num_page)
        driver.get(get_url_anphat(base_url, num_page))
        time.sleep(5)

        total_height = int(driver.execute_script("return document.body.scrollHeight"))

        for i in range(1, total_height, 100):
            driver.execute_script("window.scrollTo(0, {});".format(i))
        items = driver.find_elements_by_class_name("p-item")
        for item in items:
            link = item.find_element_by_css_selector(".p-img").get_attribute("href")
            name = item.find_element_by_css_selector(".p-name").text
            list_name.append(name)
            list_url.append(link)
        num_page += 1
    return list_name, list_url

In [ ]:
def crawl_anphat_2(driver, list_url):
    list_name = []
    list_link = []
    for url in list_url:
        driver.get(url)
        time.sleep(2)
        print("connect to: ", url)
        temp_height = int(driver.execute_script("return document.body.scrollHeight") / 10)

        for i in range(1, temp_height, 100):
            driver.execute_script("window.scrollTo(0, {});".format(i))
        temp = driver.find_elements_by_class_name("item")
        print(temp)
        if len(temp) == 0:
            continue
        for t in temp:
            temp_name = t.get_attribute("input[type='checkbox']").text
            print(temp_name)
    return list_name, list_link


In [ ]:
%cd "/content/drive/MyDrive/KHDL"
import pandas as pd
driver = webdriver.Chrome(PATH,chrome_options=chrome_options)
driver.maximize_window()

name, url = crawl_phongvu(driver, phong_vu)
# second_name, second_url = crawl_anphat_2(driver, ["https://www.anphatpc.com.vn/laptop-apple-macbook-pro-2020-mxk62sa-a-silver_id33574.html"])
data = pd.DataFrame({"Name" : name, "Link" : url})
data.to_csv("Crawl_Search_Phong_Vu.csv", sep="\t", index=None)

/content/drive/MyDrive/KHDL
Page:  1
Crawl 0 of 35
Crawl 1 of 35
Crawl 2 of 35
Crawl 3 of 35
Crawl 4 of 35
Crawl 5 of 35
Crawl 6 of 35
Crawl 7 of 35
Crawl 8 of 35
Crawl 9 of 35
Crawl 10 of 35
Crawl 11 of 35
Crawl 12 of 35
Crawl 13 of 35
Crawl 14 of 35
Crawl 15 of 35
Crawl 16 of 35
Crawl 17 of 35
Crawl 18 of 35
Crawl 19 of 35
Crawl 20 of 35
Crawl 21 of 35
Crawl 22 of 35
Crawl 23 of 35
Crawl 24 of 35
Crawl 25 of 35
Crawl 26 of 35
Crawl 27 of 35
Crawl 28 of 35
Crawl 29 of 35
Crawl 30 of 35
Crawl 31 of 35
Crawl 32 of 35
Crawl 33 of 35
Crawl 34 of 35
Page:  2
Crawl 0 of 25
Crawl 1 of 25
Crawl 2 of 25
Crawl 3 of 25
Crawl 4 of 25
Crawl 5 of 25
Crawl 6 of 25
Crawl 7 of 25
Crawl 8 of 25
Crawl 9 of 25
Crawl 10 of 25
Crawl 11 of 25
Crawl 12 of 25
Crawl 13 of 25
Crawl 14 of 25
Crawl 15 of 25
Crawl 16 of 25
Crawl 17 of 25
Crawl 18 of 25
Crawl 19 of 25
Crawl 20 of 25
Crawl 21 of 25
Crawl 22 of 25
Crawl 23 of 25
Crawl 24 of 25
error page


#Hiển thị kết quả crawl

In [ ]:
%cd "/content/drive/MyDrive/KHDL"
import pandas as pd
data_1 = pd.read_csv("Crawl_Search_An_Phat.csv", sep="\t")
data_2 = pd.read_csv("Crawl_Search_HN_CPT.csv", sep="\t")
data_3 = pd.read_csv("Crawl_Search_Phong_Vu.csv", sep="\t")
data = data_1.append(data_2)
data = data.append(data_3)
data

/content/drive/MyDrive/KHDL


,Name,Link
0,Laptop Acer Gaming Nitro 5 Eagle AN515-57-720A...,https://www.anphatpc.com.vn/laptop-acer-gaming...
1,Laptop LG Gram 2021 16Z90P-G.AH73A5 (Core i7-1...,https://www.anphatpc.com.vn/laptop-lg-gram-202...
2,Laptop Asus ROG Strix G15 G513IC-HN002T (Ryzen...,https://www.anphatpc.com.vn/laptop-asus-rog-st...
3,Laptop Lenovo V15 G2 ITL 82KB00CHVN (Core™ i5-...,https://www.anphatpc.com.vn/laptop-lenovo-v15-...
4,Laptop Asus TUF Gaming A15 FA506QM-HN016T (Ryz...,https://www.anphatpc.com.vn/laptop-asus-tuf-ga...
...,...,...
50,Laptop ASUS UX482EA-KA268T 90NB0S41-M05040 ( 1...,https://phongvu.vn/may-tinh-xach-tay-laptop-as...
51,Laptop Dell Alienware M15 Ryzen Edition R5 702...,https://phongvu.vn/may-tinh-xach-tay-laptop-de...
52,"Laptop Dell Alienware M15 R6 70262923 ( 15.6"" ...",https://phongvu.vn/may-tinh-xach-tay-laptop-de...
53,Laptop ACER Nitro 5 AN515-57-56S5 NH.QEKSV.001...,https://phongvu.vn/may-tinh-xach-tay-laptop-ac...


In [ ]:
data_1

,Name,Link
0,Laptop Acer Gaming Nitro 5 Eagle AN515-57-720A...,https://www.anphatpc.com.vn/laptop-acer-gaming...
1,Laptop LG Gram 2021 16Z90P-G.AH73A5 (Core i7-1...,https://www.anphatpc.com.vn/laptop-lg-gram-202...
2,Laptop Asus ROG Strix G15 G513IC-HN002T (Ryzen...,https://www.anphatpc.com.vn/laptop-asus-rog-st...
3,Laptop Lenovo V15 G2 ITL 82KB00CHVN (Core™ i5-...,https://www.anphatpc.com.vn/laptop-lenovo-v15-...
4,Laptop Asus TUF Gaming A15 FA506QM-HN016T (Ryz...,https://www.anphatpc.com.vn/laptop-asus-tuf-ga...
...,...,...
653,Laptop Apple Macbook Pro 16-inch MVVM2SA/A Silver,https://www.anphatpc.com.vn/laptop-apple-macbo...
654,Laptop Asus ROG ZEPHYRUS S GX502LWS-HF070T,https://www.anphatpc.com.vn/laptop-asus-rog-ze...
655,Laptop Apple Macbook Pro 2020 MWP72SA/A (Silver),https://www.anphatpc.com.vn/laptop-apple-macbo...
656,Laptop Apple Macbook Pro 2020 MWP42SA/A (Space...,https://www.anphatpc.com.vn/laptop-apple-macbo...


In [ ]:
data_2

,Name,Link
0,LAPTOP ASUS ZENBOOK UX325EA-KG363T (I5 1135G7/...,https://www.hanoicomputer.vn/laptop-asus-zenbo...
1,LAPTOP ASUS GAMING ROG FLOW 13 GV301QC-K6029T ...,https://www.hanoicomputer.vn/laptop-asus-gamin...
2,LAPTOP LENOVO THINKPAD X1 GEN 9 (20XW0076VN) (...,https://www.hanoicomputer.vn/laptop-lenovo-thi...
3,LAPTOP LENOVO LEGION 7 16ACHG6 (82N60039VN) (R...,https://www.hanoicomputer.vn/laptop-lenovo-leg...
4,LAPTOP ACER GAMING PREDATOR TRITON 300 (PT315-...,https://www.hanoicomputer.vn/laptop-acer-gamin...
...,...,...
916,LAPTOP LENOVO LEGION 5-15ACH6A (82NW003BVN) (R...,https://www.hanoicomputer.vn/laptop-lenovo-leg...
917,LAPTOP LENOVO LEGION 5-15ACH6A (82NW003CVN) (R...,https://www.hanoicomputer.vn/laptop-lenovo-leg...
918,LAPTOP DELL INSPIRON 3501 (70253897) (I5 1135G...,https://www.hanoicomputer.vn/laptop-dell-inspi...
919,LAPTOP DELL VOSTRO 3400 (70253899) (I3 1115G4/...,https://www.hanoicomputer.vn/laptop-dell-vostr...


In [ ]:
data_3